# tests

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame({
    "vp": [6,3,6,5,10], 
    "p": [4,4,2,12,23], 
    'n': [5,14,1,32,12],
    'r': [3,7,3,27,5],
    'vr': [2,6,0,18,14],
})

In [ ]:
df

In [ ]:
df.apply(lambda x:x/x.sum(),axis=1)

# Introduction

- outliers regularisation method = mean
- mean regularisation columns = [
    "PropertyType",
    'County',
    "PostcodeArea",
    'District',
    'TownCity',
]
- "isLog" : True,
- 'classify_outliers_cycle': 2

--> r2 = 0.74

## Configs

In [ ]:
config_processing = {
    # ----- default
    # """
    #     r2 = 0.6827
    #     inputs = [
    #         'OldNew' ,  
    #         'Duration' , 
    #         'PPDCategoryType',
    #         # Month
    #         *[k for k in data.columns if ('Month' in k)],
    #         # Weights
    #         *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    #         # # zscore
    #         *[k for k in data.columns if ('WeightsTransf' in k)],
    #         # PropertyType
    #         *[k for k in data.columns if ('PropertyType' in k)],
    #         # target
    #         target
    #     ]
    # """
    "default" : {
        "method" : "mean",
        "groupby" : [
            "PPDCategoryType",
            "PropertyType",
            "Duration",
            'District',
            'TownCity',
        ],
        "isLog" : False,
        'classify_outliers_cycle': 2
    },
    # ----- v1
    # """
    #     r2 = 0.6813    
    #     inputs = [
    #         'OldNew' ,  
    #         'Duration' , 
    #         'PPDCategoryType',
    #         # Month
    #         *[k for k in data.columns if ('Month' in k)],
    #         # Weights
    #         *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    #         # PropertyType
    #         *[k for k in data.columns if ('PropertyType' in k)],
    #         # target
    #         target
    #     ]
    # """
    "version 1" : {
        "method" : "mean",
        "groupby" : [
            "PropertyType",
            "Duration",
            'County',
            'District',
            'TownCity',
        ],
        "isLog" : False,
        'classify_outliers_cycle': 2
    },
    # ----- v2 
    # """
    #     r2 = 0.6964    
    #     inputs = [
    #         'OldNew' ,  
    #         'Duration' , 
    #         'PPDCategoryType',
    #         # Month
    #         *[k for k in data.columns if ('Month' in k)],
    #         # Weights
    #         *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    #         # PropertyType
    #         *[k for k in data.columns if ('PropertyType' in k)],
    #         # target
    #         target
    #     ]
    # """
    "version 2" : {
        "method" : "mean",
        "groupby" : [
            # --- current model
            # "PropertyType",
            # 'County',
            # "PostcodeArea",
            # 'District',
            # 'TownCity',
            # --- test,
            "PropertyType",
            'County',
            'TownCity',
            "PostcodeArea",
            'District',
            
        ],
        "isLog" : True,
        'classify_outliers_cycle': 2
    },
    # ----- v3 
    # """
    #     r2 = ??    
    #     inputs = [
    #         'OldNew' ,  
    #         'Duration' , 
    #         'PPDCategoryType',
    #         # Month
    #         *[k for k in data.columns if ('Month' in k)],
    #         # Weights
    #         *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    #         # PropertyType
    #         *[k for k in data.columns if ('PropertyType' in k)],
    #         # target
    #         target
    #     ]
    # """
    "version 3" : {
        "method" : "mean",
        "groupby" : [
            "PropertyType",
            "Duration",
            'County',
            "PostcodeArea",
            'District',
            'TownCity',
            
        ],
        "isLog" : False,
        'classify_outliers_cycle': 2
    },
}

In [ ]:
choosed_version = "version 2"
# ----- define elements
outliers_regularisation_method = config_processing[choosed_version]["method"]
groupbycolumns = config_processing[choosed_version]["groupby"]
logoutlierfence= config_processing[choosed_version]["isLog"]
classify_outliers_cycle= config_processing[choosed_version]["classify_outliers_cycle"]

## Paths & Folders

- for Model directory for saving and loading is stored in "model_dir_location"

In [1]:
model_name = 'model_xgboost_withIncomeArea'

In [2]:
model_dir_location = f'../models/{model_name}'

# Load data

In [3]:
# %load_ext cudf.pandas
import math 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import scipy as sp
pd.set_option('float_format', '{:f}'.format)
plt.style.use('ggplot')

import pickle
import json
from matplotlib import style 
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [4]:
columns = [
    'Id',
    'Price',
    'TransferDate',
    'Postcode',
    'PropertyType',
    'OldNew',
    'Duration',
    'PAON',
    'SAON',
    'Street',
    'Locality' ,
    'TownCity',
    'District',
    'County',
    'PPDCategoryType',
    'RecordStatus',
]
houses_loaded = pd.read_csv('../data/pp-2021.csv',names=columns)

# Functions

## visualization

In [5]:
def find_df_name(df):
   name = [name for name, obj in globals().items() if id(obj) == id(df)]
   return name[0] if name else None

def bar_catg_feature(col_name,df_data):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
    # data
    df_feature = df_data.loc[:,[col_name,"Price"]]

    df_grouped = df_feature.groupby(col_name)["Price"].mean()
    # first plot
    sns.barplot(
        x=df_grouped.index, 
        y=df_grouped.values, 
        ax=axes[0]
    )
    # second plot
    values = df_feature[col_name].value_counts(dropna=False)
    sns.barplot(
        x=values.index, 
        y=values.values,
        ax=axes[1]
    )
    # Add titles
    axes[0].set_title(f'Average Price by {col_name} (data={find_df_name(df_data)})')
    axes[1].set_title(f'distribution of {col_name} (data={find_df_name(df_data)})')
    # show
    plt.tight_layout()
    plt.show()

def bar_catg_pairfeature(colname1,colname2,colname3='Price',df_data=None):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 6))
    # data
    df_feature = df_data.loc[:,[colname1,colname2,colname3]]
    sns.barplot(
        data = df_feature,
        x=colname2, 
        y=colname3,
        hue=colname1,
        ax=axes
    )
    # Add titles
    axes.set_title(f'Average Price by {colname1}, {colname2} (data={find_df_name(df_data)})')
    # show
    plt.tight_layout()
    plt.show()

def bar_geo_feature(colname1,colname2='Price',df_data = None,ascending=False,method='mean'):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 12))
    # data
    if method=='mean':
        df_feature = df_data.groupby(colname1)[colname2].mean().sort_values(ascending=ascending)[:50]
    elif method=='median':
        df_feature = df_data.groupby(colname1)[colname2].median().sort_values(ascending=ascending)[:50]
    sns.barplot(
        x=df_feature.index, 
        y=df_feature.values,
        ax=axes
    )
    # Add titles
    axes.tick_params(axis='x', rotation=90)
    axes.set_title(f'{method} of {colname2} by {colname1} (data={find_df_name(df_data)})')
    # show
    # plt.tight_layout()
    plt.show()

def box_geo_feature(colname1,colname2='Price',df_data = None,method='mean'):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 6))
    if method=='mean':
        df_grouped_mean = df_data.groupby(colname1)[colname2].mean().sort_values(ascending=False)
    elif method=='median':
        df_grouped_mean = df_data.groupby(colname1)[colname2].median().sort_values(ascending=False)
    sns.boxplot(df_grouped_mean)
    plt.title(f'Distribution of {method} of {colname2} by {colname1} (data={find_df_name(df_data)})')
    plt.show()

def price_distribution(df,bins=100,kind='bar',transform_log=0):
    if transform_log==0:
        price = df['Price']  
    else:
        price = df['Price']
        for i in range(transform_log):
            price = np.log1p(price)
    price.value_counts(bins=bins).sort_index().plot(kind=kind,figsize=(20,16))
    # show
    plt.tight_layout()
    plt.show()

def price_boxplot(df,col_name=None,isLog=False):
    kwargs = {
        'y' : np.log(df['Price']) if isLog else df['Price']
    }
    if col_name: kwargs['x'] = df[col_name]  
    ax = sns.boxplot(**kwargs)
    title = f'Price by {col_name}' if col_name!= None else 'Price' 
    ax.set_title(title + f' :(log={isLog},data={find_df_name(df)})',fontsize=10)
    plt.show()

def price_scatterplot(df,col_name=None,isColDivided=False,isLog=False):
    
    if isColDivided:
        # -- inputs
        vv = df[col_name].value_counts().index
        ncols = 2
        nrows = len(vv)//ncols + len(vv)%ncols
        # -- canvas
        fig, ax = plt.subplots(
            nrows = nrows,
            ncols = ncols, 
            figsize=(10*ncols,8*nrows)
        )
        # -- draw
        i=j=0
        for ptype in vv:
            dd = df.loc[df[col_name] == ptype,['Price',col_name]]
            if nrows == 1:
                sns.scatterplot(
                    x = dd[col_name],
                    y = np.log(dd.Price) if isLog else dd.Price,
                    ax = ax[j]
                ).set(title=f'{col_name} ({ptype})')
                # ax[j].plot(dd)
                # ax[j].title.set_text(f'{col_name}({ptype})')
            else : 
                sns.scatterplot(
                    x = dd[col_name],
                    y = np.log(dd.Price) if isLog else dd.Price,
                    ax=ax[i, j]
                ).set(title=f'{col_name} ({ptype})')
                # ax[i, j].plot(dd)
                # ax[i, j].title.set_text(f'{col_name}({ptype})')
            if j == 1:
                i += 1
                j=0
            else: j+=1
        fig.suptitle(f'Price by {col_name} :(log={isLog},data={find_df_name(df)})')
    else:
        kwargs = {
            'y' : np.log(df['Price']) if isLog else df['Price']
        }
        if col_name: kwargs['x'] = df[col_name] 
        ax = sns.scatterplot(**kwargs)
        title = f'Price by {col_name}' if col_name!= None else 'Price' 
        ax.set_title(title + f' :(log={isLog},data={find_df_name(df)})',fontsize=10)
    plt.show()

def visualise_grps_bycol(df,col_name,ncols=2):
    vv = df[col_name].value_counts().index
    nrows = len(vv)//ncols + len(vv)%ncols
    fig , ax = plt.subplots(
        nrows = nrows,
        ncols = ncols,
        figsize=(10*ncols,8*nrows)
    )
    i = j = 0
    for ptype in vv:
        print(f"{ptype} : ({i},{j})")
        dd = df.loc[df[col_name] == ptype,'Price']
        if nrows == 1:
            ax[j].plot(dd)
            ax[j].title.set_text(f'{col_name}({ptype})')
        else : 
            ax[i, j].plot(dd)
            ax[i, j].title.set_text(f'{col_name}({ptype})')
        if j == 1:
            i += 1
            j=0
        else: j+=1
    plt.show()

def visualise_grps(df,col_name,ncols=2):
    bar_catg_feature(col_name,df_data = df)
    box_geo_feature(col_name,df_data = df)
    price_boxplot(df,col_name=col_name,isLog=False)
    price_boxplot(df,col_name=col_name,isLog=True)
    visualise_grps_bycol(df,col_name,ncols=ncols)

def visualise_global(df,ncols=2,bins=100,grp_columns = [],geo_col = [],scatter_col=[]):
    """
        grp_columns = ["PropertyType","Duration","OldNew"]
        geo_col = ["County","District","TownCity"]
    """
    # -- Price
    print("###################################################### Price ######################################################")
    price_distribution(df,bins=bins,kind='bar')
    price_boxplot(df,col_name=None,isLog=False)
    price_boxplot(df,col_name=None,isLog=True)
    
    # -- grp_columns
    for col in grp_columns:
        print(f"###################################################### {col} ######################################################")
        visualise_grps(df,col,ncols)
    
    # -- geo_col
    for col in geo_col:
        print(f"###################################################### {col} ######################################################")
        box_geo_feature(col,df_data = df)
        bar_geo_feature(col,colname2='Price',df_data = df,ascending=False,method='mean')
    
    # -- scatterplot
    for col in scatter_col:
        print(f"###################################################### {col} ######################################################")
        price_scatterplot(df,col_name=col,isColDivided=False,isLog=False)

## aggs & transformation

In [6]:
# function to add to JSON
def write_json(new_data, filename='features_data/data.json',isNew=False):
    if isNew:
        with open(filename, 'x') as f:
            json.dump({"data":new_data},f)
    else:
        with open(filename,'r+') as file:
              # First we load existing data into a dict.
            file_data = json.load(file)
            # Join new_data with file_data inside emp_details
            file_data["data"].update(new_data)
            # Sets file's current position at offset.
            file.seek(0)
            # convert back to json.
            json.dump(file_data, file, indent = 4)

def load_json(data_name=None, filename='features_data/data.json'):
    with open(filename,'r') as file:
        file_data = json.load(file)
        data = file_data["data"] if data_name==None else file_data["data"][data_name]
    return data

def fix_dfHouses(df):
    def clean(x):
        if type(x)==str:
            return x.strip()
        elif x==None : return np.nan
        elif math.isnan(x): return np.nan
        else: return x
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(clean)
    # PropertyType : D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other
    df.loc[df['PropertyType']=='T','PropertyType'] = 'Terraced'
    df.loc[df['PropertyType']=='S','PropertyType'] = 'SemiDetached'
    df.loc[df['PropertyType']=='D','PropertyType'] = 'Detached'
    df.loc[df['PropertyType']=='F','PropertyType'] = 'Flat'
    df.loc[df['PropertyType']=='O','PropertyType'] = 'Other'
    # OldNew : Y = New, N = Old
    df.loc[df['OldNew']=='Y','OldNew'] = 'New'
    df.loc[df['OldNew']=='N','OldNew'] = 'Old'
    # Duration : F = Freehold, L= Leasehold
    df.loc[df['Duration']=='F','Duration'] = 'Freehold'
    df.loc[df['Duration']=='L','Duration'] = 'Leasehold'
    # PPDCategoryType : A = StandardPrice (Standard Price Paid entry) ,  B = AdditionalPrice (Additional Price Paid)
    df.loc[df['PPDCategoryType']=='A','PPDCategoryType'] = 'StandardPrice'
    df.loc[df['PPDCategoryType']=='B','PPDCategoryType'] = 'AdditionalPrice'
    # TransferDate --> Day, Month
    df['TransferDate'] = pd.to_datetime(df['TransferDate'])
    df['TransferMonth'] = df['TransferDate'].dt.month
    df['TransferDay'] = df['TransferDate'].dt.day
    return df

def categorise(col,df_data,weight_func = lambda x:x):
    cat_col = df_data.groupby(col)['Price'].mean()
    # Calculate the quartiles:
    q25 = np.percentile(cat_col, 25)
    q75 = np.percentile(cat_col, 75)
    max_q = min(cat_col.max(),q75+1.5*(q75-q25))
    min_q = max(cat_col.min(),q25-1.5*(q75-q25))
    # caterg
    def col_cat(x,min_q,q25,q75,max_q):
        if x["Price"] < min_q : return "Very Poor"
        elif (x["Price"] >= min_q) and (x["Price"] <= q25) : return "Poor"
        elif (x["Price"] > q25) and (x["Price"] <= q75) : return "Normal"
        elif(x["Price"] > q75) and (x["Price"] <= max_q) : return "Rich"
        else: return "Very Rich"
    def coef_calc(x,coltype,coef):
        return coef_types[x[coltype]]*x['counts']
    # def cat_dfdata(df,col,weights):
    #     df_cp = df.copy()
    #     df_cp.loc[:,col+'Weights'] = df_cp.apply(lambda x:weights[x[col]],axis=1)
    #     return df_cp
    # caterg
    coltype = col+'Type'
    houses_categorised = pd.DataFrame(df_data.apply(lambda x: col_cat(x,min_q=min_q,q25=q25,q75=q75,max_q=max_q),axis=1),columns = [coltype])
    houses_categorised = pd.concat([df_data,houses_categorised], axis=1)
    houses_categorised_byType = houses_categorised.loc[:,[col,coltype]].groupby([col,coltype]).agg(
        counts = pd.NamedAgg(column=coltype, aggfunc=lambda x:x.count()),
    )
    coef_types_weights = {
        'Normal' : 0,
        'Poor': -2,
        'Rich': 2,
        'Very Poor': -4,
        'Very Rich': 4,
    }
    coef_types = {k:weight_func(v) for k,v in coef_types_weights.items()}
    houses_categorised_byType = houses_categorised_byType.groupby(level=[col],group_keys=False).apply(lambda x:x/x.sum()).reset_index()
    houses_categorised_byType['weights'] = houses_categorised_byType.apply(lambda x: coef_calc(x,coltype,coef_types),axis=1)
    categories_weights = houses_categorised_byType.groupby(col)['weights'].sum()
    
    return {
        'categories_weights':categories_weights,
        col+'Weights': df_data.apply(lambda x:categories_weights[x[col]],axis=1)
    }

def classify_outliers(df_data,colname='Price',by_log=False):
    # target
    if not by_log:
        target_col = df_data[colname]
        target_metrics = target_col.describe()
    else:
        target_col = np.log(df_data[colname].astype(float))
        target_metrics = target_col.describe()
    # metrics 
    price_q25= target_metrics["25%"]
    price_q50= target_metrics["50%"]
    price_q75= target_metrics["75%"]
    price_IQR = price_q75 - price_q25
    price_max= min(target_metrics["max"],price_q75+1.5*price_IQR)
    price_min= max(target_metrics["min"],price_q25-1.5*price_IQR)
    mask = np.logical_and((target_col <= price_max), (target_col >= price_min))
    normal_PriceWise_df = df_data.loc[mask,:]
    expensive_PriceWise_df = df_data.loc[~mask,:]
    return {
        "q25":price_q25,
        "q50":price_q50,
        "q75":price_q75,
        "IQR":price_IQR,
        "max":price_max,
        "min":price_min,
        "normal_PriceWise_df":normal_PriceWise_df,
        "expensive_PriceWise_df":expensive_PriceWise_df
    }

def regulate_outliers(df_data, method='mean', groupby=None,bothsides=False,maxmin=False,df_reference=pd.DataFrame()):
    """
        1- expl:
            maxmin: 
                - x>outliser_bar_max ---> x=outliser_bar_max
                - x<outliser_bar_min ---> x=outliser_bar_min
            bothsides:
                - (x>outliser_bar_max) or (x<outliser_bar_min) ---> x=groupby(method)
            groupby:
                - name of column of group by
                - list or str
            
        2- Notes:
            - "maxmin" is prioritised over "bothsides" 
    """
    data = df_data.copy()
    df_data_reference = df_reference.copy() if not df_reference.empty else data
    if groupby != None:
        df_data_grps = df_data_reference.groupby(groupby).agg(
            min = pd.NamedAgg(column='Price',aggfunc='min'),
            outlier_bar_Min = pd.NamedAgg(column='Price',aggfunc=lambda x:max(min(x),np.percentile(x,25) - 1.5*(np.percentile(x,75)-np.percentile(x,25)))),
            q25 = pd.NamedAgg(column='Price',aggfunc=lambda x:np.percentile(x,25)),
            q50 = pd.NamedAgg(column='Price',aggfunc=lambda x:np.percentile(x,50)),
            q75 = pd.NamedAgg(column='Price',aggfunc=lambda x:np.percentile(x,75)),
            outlier_bar_Max = pd.NamedAgg(column='Price',aggfunc=lambda x:min(max(x),np.percentile(x,75) + 1.5*(np.percentile(x,75)-np.percentile(x,25)))),
            max = pd.NamedAgg(column='Price',aggfunc='max'),
            median = pd.NamedAgg(column='Price',aggfunc='median'),
            mean = pd.NamedAgg(column='Price',aggfunc='mean'),
        )
        def fix_outliers(x,df_data_grps,method,groupby,bothsides):
            # print(x)
            if not (tuple(x[groupby]) in df_data_grps.index): return np.nan 
            s_x = df_data_grps.loc[tuple(x[groupby]),:]
            if x['Price']<s_x['outlier_bar_Min']: 
                if maxmin or (not bothsides): return s_x['outlier_bar_Min'] 
                else: return s_x[method]
            elif x['Price']>s_x['outlier_bar_Max']: 
                if maxmin: return s_x['outlier_bar_Max'] 
                else: return s_x[method]
            else : return x['Price']
        kwargs = {
            'df_data_grps':df_data_grps,
            'method':method,
            'groupby':groupby,
            'bothsides':bothsides,
        } 
    else:
        metric_method = df_data_reference.Price.agg(method)
        df_data_grps = df_data_reference.Price.describe()
        
        def fix_outliers(x,df_data_grps,metric_method,bothsides):
            # -- metrics
            min_p = df_data_grps['min']
            max_p = df_data_grps['max']
            q25_p = df_data_grps['25%']
            q75_p = df_data_grps['75%']
            bar_max = min(max_p,q75_p + 1.5*(q75_p-q25_p))
            bar_min = max(min_p,q25_p - 1.5*(q75_p-q25_p))
            # -- regulate
            if x['Price']<bar_min:
                if maxmin or (not bothsides): return bar_min 
                else: return metric_method
            elif x['Price']>bar_max: 
                if maxmin: return bar_max 
                else: return metric_method
            else : return x['Price']
        kwargs = {
            'df_data_grps':df_data_grps,
            'metric_method':metric_method,
            'bothsides':bothsides
        }
    return data.apply(lambda x:fix_outliers(x,**kwargs),axis=1)

def stratified_sampling(df,colname,frac,random_state=None):
    sampling_options = {
        "frac":frac
    }
    if random_state!= None : sampling_options['random_state']=random_state
    return df.groupby(colname, group_keys=False).apply(lambda x: x.sample(**sampling_options))

def transform_feat(df,cols=[],func_transform=lambda x:x):
    df_cp = df.copy()
    for col in cols:
        df_cp[col+"Transf"] = df_cp[col].apply(func_transform)
    return df_cp

# Discovery

## Exploration

In [ ]:
houses_loaded.info()

In [ ]:
houses_loaded.describe()

## Cleaning

In [7]:
houses_loaded = fix_dfHouses(houses_loaded)

In [ ]:
cat_features = [
    'PropertyType',
    'OldNew',
    'Duration',
    'TownCity',
    'District',
    'County',
    'PPDCategoryType',
    'RecordStatus',
]
for col in cat_features:
    print("Number of categories of Feature '{0}' is : {1} ".format(col, len(houses_loaded[col].unique())))
    print(houses_loaded[col].value_counts(dropna=False).index.tolist())
    print('--------------------------------------------------------------------------------------------')

In [8]:
houses_loaded.head()

,Id,Price,TransferDate,Postcode,PropertyType,OldNew,Duration,PAON,SAON,Street,Locality,TownCity,District,County,PPDCategoryType,RecordStatus,TransferMonth,TransferDay
0,{CB0035E6-3546-58AE-E053-6B04A8C091AF},630000,2021-04-29,BN13 3AH,Detached,Old,Freehold,3,NaN,HIGHLANDS CLOSE,NaN,WORTHING,WORTHING,WEST SUSSEX,StandardPrice,A,4,29
1,{CB0035E6-3547-58AE-E053-6B04A8C091AF},477000,2021-05-10,BN16 2PQ,Detached,Old,Freehold,10,NaN,CHAUCER AVENUE,RUSTINGTON,LITTLEHAMPTON,ARUN,WEST SUSSEX,StandardPrice,A,5,10
2,{CB0035E6-3548-58AE-E053-6B04A8C091AF},370000,2021-05-06,BN2 4HZ,Terraced,Old,Freehold,6,NaN,DARTMOUTH CLOSE,NaN,BRIGHTON,BRIGHTON AND HOVE,BRIGHTON AND HOVE,StandardPrice,A,5,6
3,{CB0035E6-3549-58AE-E053-6B04A8C091AF},462500,2021-04-28,BN2 0GP,Terraced,Old,Freehold,18,NaN,STANLEY STREET,NaN,BRIGHTON,BRIGHTON AND HOVE,BRIGHTON AND HOVE,StandardPrice,A,4,28
4,{CB0035E6-354A-58AE-E053-6B04A8C091AF},433000,2021-08-06,BN8 4LS,Detached,Old,Freehold,11,NaN,POWELL ROAD,NEWICK,LEWES,LEWES,EAST SUSSEX,StandardPrice,A,8,6


In [22]:
houses_loaded.loc[houses_loaded.Postcode== 'BRIGHTON',:].Postcode.value_counts()

BN2 3PJ     15
BN1 4QG     14
BN2 0EE     11
BN1 8EQ     10
BN2 8RQ     10
            ..
BN42 4YB     1
BN41 1PW     1
BN1 1NH      1
BN42 4HT     1
BN45 7EB     1
Name: Postcode, Length: 1944, dtype: int64

## Visualisation

In [ ]:
visualise_global(
    houses_loaded,
    grp_columns = ["PropertyType","Duration","OldNew"],
    geo_col = ["County","District","TownCity"]
)

# Preprocessing

### PostcodeArea

In [ ]:
regex_postcode = r'(\S+)'     #r'([a-z]+)'
houses_loaded['PostcodeArea'] = houses_loaded.Postcode.str.extract(regex_postcode,flags=re.I)

In [ ]:
mask_pc = houses_loaded.PostcodeArea.isna()
town_city_PostCodesArea = houses_loaded.groupby('TownCity')["PostcodeArea"].apply(lambda x:pd.Series(list(set(x))).dropna().tolist()[0])
houses_loaded.loc[mask_pc,'PostcodeArea']= houses_loaded.loc[(mask_pc),['PostcodeArea',"TownCity"]].apply(lambda x:town_city_PostCodesArea[x["TownCity"]],axis=1)

In [ ]:
houses_loaded.PostcodeArea.value_counts(dropna=True)

### normal_PriceWise_houses && expensive_PriceWise_houses

- 'logoutlierfence' is declared in "introduction" section

#### Outlier Treatement

In [ ]:
normal_PriceWise_houses = houses_loaded.copy()
for i in range(classify_outliers_cycle):
    classified_houses = classify_outliers(
        df_data=normal_PriceWise_houses,
        colname='Price',
        by_log = logoutlierfence
    )
    expensive_PriceWise_houses = classified_houses["expensive_PriceWise_df"]
    normal_PriceWise_houses = classified_houses["normal_PriceWise_df"]
    logoutlierfence = not logoutlierfence

#### Visualization

In [ ]:
grp_columns = ["PropertyType","Duration","OldNew"]
geo_col = ["County","District","TownCity","PostcodeArea"]

visualise_global(
    normal_PriceWise_houses,
    bins=100,
    grp_columns=grp_columns,
    geo_col=geo_col
)

In [ ]:
visualise_global(
    expensive_PriceWise_houses,
    grp_columns = ["PropertyType","Duration","OldNew"],
    geo_col = ["County","District","TownCity"],
    bins=100
)

### houses_regulated

- 'outliers_regularisation_method', 'groupbycolumns' are declared in "introduction" section

#### Regulate "expensive_PriceWise_houses" and merge it with "normal_PriceWise_houses"

In [ ]:
reg_expensive_PriceWise_houses = expensive_PriceWise_houses.copy()
reg_expensive_PriceWise_houses['Price'] = regulate_outliers(
    df_data = expensive_PriceWise_houses, 
    method=outliers_regularisation_method, 
    groupby = groupbycolumns,
    bothsides=False,
    maxmin=False,
    df_reference=normal_PriceWise_houses
)

In [ ]:
houses_regulated = pd.concat([normal_PriceWise_houses,reg_expensive_PriceWise_houses], axis=0)

In [ ]:
houses_regulated = houses_regulated.loc[houses_regulated.Price.notna(),:]

In [ ]:
houses_regulated.reset_index(drop=True,inplace=True)

In [ ]:
houses_regulated['Price'] = regulate_outliers(
    df_data = houses_regulated, 
    method=outliers_regularisation_method, 
    groupby = groupbycolumns,
    bothsides=False,
    maxmin=False
)

#### Visualization

In [ ]:
houses_regulated.describe()

In [ ]:
visualise_global(
    houses_regulated,
    grp_columns = ["PropertyType","Duration","OldNew"],
    geo_col = ["County","District","TownCity"]
)

In [ ]:
price_distribution(houses_regulated,bins=100,kind='bar',transform_log=1)

### Houses

In [ ]:
houses = houses_regulated.copy()
houses.reset_index(drop=True,inplace=True)

In [ ]:
houses.isna().sum()

#### "Weights" Feature

In [ ]:
weight_func = lambda x:x

##### Categorise Features weights

###### County

In [ ]:
categorised_counties_data = categorise(
    col="County",
    df_data=houses,
    weight_func = weight_func
)
houses.loc[:,'CountyWeights'] = categorised_counties_data['CountyWeights']
counties_weights = categorised_counties_data['categories_weights']

In [ ]:
data_counties_weights = {
    "counties_weights" : counties_weights.to_dict()
}
write_json(data_counties_weights,isNew=True)

###### District

In [ ]:
categorised_districts_data = categorise(
    col="District",
    df_data=houses,
    weight_func = weight_func
)
houses.loc[:,'DistrictWeights'] = categorised_districts_data['DistrictWeights']
districts_weights = categorised_districts_data['categories_weights']

In [ ]:
data_districts_weights = {
    "districts_weights" : districts_weights.to_dict()
}
write_json(data_districts_weights)

###### TownCity

In [ ]:
categorised_cities_data = categorise(
    col="TownCity",
    df_data=houses,
    weight_func = weight_func
)
houses.loc[:,'TownCityWeights'] = categorised_cities_data['TownCityWeights']
cities_weights = categorised_cities_data['categories_weights']

In [ ]:
data_cities_weights = {
    "cities_weights" : cities_weights.to_dict()
}
write_json(data_cities_weights)

###### Postcode

In [ ]:
categorised_PostcodeArea_data = categorise(
    col="PostcodeArea",
    df_data=houses,
    weight_func = weight_func
)
houses.loc[:,'PostcodeAreaWeights'] = categorised_PostcodeArea_data['PostcodeAreaWeights']
PostcodeArea_weights = categorised_PostcodeArea_data['categories_weights']

In [ ]:
data_PostcodeArea_weights = {
    "PostcodeArea_weights" : PostcodeArea_weights.to_dict()
}
write_json(data_PostcodeArea_weights)

##### Load Features weights

In [ ]:
weights_data = load_json()
weights_dicts = {
    'PostcodeArea' : weights_data["PostcodeArea_weights"],
    'TownCity' : weights_data["cities_weights"],
    'District' : weights_data["districts_weights"],
    'County' : weights_data["counties_weights"]
}

for k,v in weights_dicts.items():
    houses.loc[:,k+'Weights'] = houses.apply(lambda x:v[x[k]],axis=1)

##### Visualisation

In [ ]:
visualise_global(
    houses,
    bins=100,
    scatter_col=[
        "CountyWeights",
        "DistrictWeights",
        "TownCityWeights",
        "PostcodeAreaWeights"
    ]
)

#### Feature Transformation

In [ ]:
houses_featured = houses.copy()
col_scatter = []

##### 'transformation_func' for each column

In [ ]:
scatter_col = {
    "CountyWeights":lambda x: x**3,   #sigmoid : transformation_func = lambda x: sp.special.expit(x)
    "DistrictWeights":lambda x: x**3,
    "TownCityWeights":lambda x: x**3,
    "PostcodeAreaWeights":lambda x: x**3
}
col_scatter_list = scatter_col if type(scatter_col)==list else scatter_col.keys()
col_scatter = [col+'Transf' for col in col_scatter_list]
for k,v in scatter_col.items():
    houses_featured = transform_feat(
        df=houses_featured,
        cols=[k],
        func_transform=v
    )

##### get 'income_area' features from api based on 'postcode'

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_incomeByPostcode(x):
    if type(x) == str:
        postcode_payload = x.strip().replace(r"\s+","+")
        url = f"https://townnames.co.uk/search/postcode-income-results?search={postcode_payload}"
        # scrap data
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            table = soup.find("table",{"id":"myTable"})
            tbody = table.find("tbody")
            data_list = [i.find_all("td")[1].text for i in tbody.find_all("tr")]
        # get area_income
            regexp = re.compile(r"\d+")
            data = [int(re.search(regexp,income).group()) for income in data_list]
            if len(data)==0: return np.nan
            return round(sum(data) / len(data), 2) #f"{sum(data) / len(data):.2f}" #float("%.2f"%(sum(data) / len(data)))
    elif math.isnan(x): return np.nan

def postcodearea_averageincome(x,postcodes_income):
    return np.nan if not (x in postcodes_income) else postcodes_income[x]

def fix_incomearea(x,area_incomes_mean):
    if not math.isnan(x["area_income"]): return x["area_income"]
    elif not math.isnan(area_incomes_mean[x["postcode_group"]]) : return area_incomes_mean[x["postcode_group"]] 
    else: return area_incomes_mean.mean()

###### option 1 : Scrap data

In [ ]:
df_postcodes_income = pd.DataFrame(houses_featured['PostcodeArea'].value_counts().index,columns= ["postcode"])
df_postcodes_income["area_income"] = df_postcodes_income["postcode"].apply(get_incomeByPostcode)

In [ ]:
postcodes_income = df_postcodes_income.copy()

In [ ]:
postcodes_income['postcode_group'] = postcodes_income.postcode.str.extract(r'([a-z]+)',flags=re.I)

In [ ]:
area_incomes_mean = postcodes_income.groupby('postcode_group')["area_income"].mean()
postcodes_income["fixed_income"] = postcodes_income.apply(lambda x:fix_incomearea(x,area_incomes_mean=area_incomes_mean),axis=1)

In [ ]:
postcodes_income_dict = postcodes_income.set_index('postcode').to_dict()["fixed_income"]

In [ ]:
data_area_income = {
    "income_ByPostcode" : postcodes_income_dict
}
write_json(data_area_income)

###### option 2 : load data

In [ ]:
postcodes_income_dict = load_json(data_name="income_ByPostcode")

###### "income_area" feature

In [ ]:
houses_featured['income_area'] = houses_featured['PostcodeArea'].apply(lambda x: postcodearea_averageincome(x,postcodes_income=postcodes_income_dict))

In [ ]:
houses_featured['income_area'].value_counts(dropna=False)

###### "income_area" feature zscore

In [ ]:
from scipy import stats
houses_featured['income_area_zscore'] = stats.zscore(houses_featured['income_area'])

##### Target : Price

In [ ]:
# f-1 : np.expm1(np.log1p(houses_featured.Price))
houses_featured['Price'] = np.log1p(houses_featured.Price)

##### Visualization

In [ ]:
houses_featured

In [ ]:
visualise_global(
    houses_featured,
    bins=100,
    scatter_col=col_scatter
)

## Data Encoding

### houses_selected

In [ ]:
selected_features = [
    'Price',
    "PropertyType",
    "OldNew",
    "Duration",
    "PPDCategoryType",
    "TransferMonth",
    "TransferDay",
    "CountyWeights",
    "DistrictWeights",
    "TownCityWeights",
    "PostcodeAreaWeights",
    *[i for i in houses_featured.columns if 'income' in i],
    *col_scatter
]
houses_selected = houses_featured.loc[:,selected_features].copy()
houses_selected.reset_index(drop=True,inplace=True)

In [ ]:
houses_selected

### houses_encoded

In [ ]:
houses_encoded = houses_selected.copy()

#### Label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

##### PPDCategoryType

In [ ]:
PPDCategoryType_encoder_dir = f'{model_dir_location}/label-encoders/'+'PPDCategoryType.npy'

###### Save

In [ ]:
PPDCategoryType_encoder = LabelEncoder()
PPDCategoryType_encoder.fit(houses_encoded['PPDCategoryType'])
np.save(PPDCategoryType_encoder_dir, PPDCategoryType_encoder.classes_)

In [ ]:
houses_encoded['PPDCategoryType'] = PPDCategoryType_encoder.transform(houses_encoded['PPDCategoryType'])

###### Load

In [ ]:
PPDCategoryType_encoder = LabelEncoder()
PPDCategoryType_encoder.classes_ = np.load(PPDCategoryType_encoder_dir,allow_pickle=True)

In [ ]:
houses_encoded['PPDCategoryType'] = PPDCategoryType_encoder.transform(houses_encoded['PPDCategoryType'])

In [ ]:
# PPDCategoryType_encoder.inverse_transform(houses_encoded['PPDCategoryType'])

##### OldNew

In [ ]:
OldNew_encoder_dir = f'{model_dir_location}/label-encoders/'+'OldNew.npy'

###### Save

In [ ]:
OldNew_encoder = LabelEncoder()
OldNew_encoder.fit(houses_encoded['OldNew'])
np.save(OldNew_encoder_dir, OldNew_encoder.classes_)

In [ ]:
houses_encoded['OldNew'] = OldNew_encoder.transform(houses_encoded['OldNew'])

###### Load

In [ ]:
OldNew_encoder = LabelEncoder()
OldNew_encoder.classes_ = np.load(OldNew_encoder_dir,allow_pickle=True)

In [ ]:
houses_encoded['OldNew'] = OldNew_encoder.transform(houses_encoded['OldNew'])

##### Duration

In [ ]:
Duration_encoder_dir = f'{model_dir_location}/label-encoders/'+'Duration.npy'

###### Save

In [ ]:
Duration_encoder = LabelEncoder()
Duration_encoder.fit(houses_encoded['Duration'])
np.save(Duration_encoder_dir, Duration_encoder.classes_)

In [ ]:
houses_encoded['Duration'] = Duration_encoder.transform(houses_encoded['Duration'])

###### Load

In [ ]:
Duration_encoder = LabelEncoder()
Duration_encoder.classes_ = np.load(Duration_encoder_dir,allow_pickle=True)

In [ ]:
houses_encoded['Duration'] = Duration_encoder.transform(houses_encoded['Duration'])

#### OneHotEncoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_dir = f'{model_dir_location}/onhot-encoders/'

##### PropretyType

In [ ]:
PropertyType_ohe_dir = f'{onehot_encoder_dir}/PropertyType_ohe'

###### Save

In [ ]:
PropertyType_ohe = OneHotEncoder(sparse_output=False) # feature_name_combiner=lambda x,y:str(x)+"_"+str(y)
PropertyType_ohe.fit(houses_encoded[["PropertyType"]])
with open(PropertyType_ohe_dir, "wb") as f: 
    pickle.dump(PropertyType_ohe,f)

###### Load

In [ ]:
with open(PropertyType_ohe_dir, 'rb') as f:
    PropertyType_ohe = pickle.load(f)

###### Transform

In [ ]:
PropertyType_features = PropertyType_ohe.transform(houses_encoded[["PropertyType"]])
df_PropertyType_features = pd.DataFrame(
    PropertyType_features, 
    columns = PropertyType_ohe.get_feature_names_out()
)
df_PropertyType_features

In [ ]:
houses_encoded = pd.concat([houses_encoded,df_PropertyType_features], axis=1)

##### Month

In [ ]:
Month_ohe_dir = f'{onehot_encoder_dir}/Month_ohe'

###### Save

In [ ]:
Month_ohe = OneHotEncoder(sparse_output=False) # feature_name_combiner=lambda x,y:str(x)+"_"+str(y)
Month_ohe.fit(houses_encoded[["TransferMonth"]])
with open(Month_ohe_dir, "wb") as f: 
    pickle.dump(Month_ohe,f)

###### Load

In [ ]:
with open(Month_ohe_dir, 'rb') as f:
    Month_ohe = pickle.load(f)

###### Transform

In [ ]:
Month_features = Month_ohe.transform(houses_encoded[["TransferMonth"]])
df_Month_features = pd.DataFrame(
    Month_features, 
    columns = Month_ohe.get_feature_names_out()
)
df_Month_features

In [ ]:
houses_encoded = pd.concat([houses_encoded,df_Month_features], axis=1)

#### fixing "houses_encoded"

In [ ]:
houses_encoded.drop(
    [
        "PropertyType",
        "TransferMonth",
        "TransferDay"
    ],
    axis=1,
    inplace=True
)

In [ ]:
input_features_encoded = [k for k in houses_encoded.columns if not (k in ("Price",'income_area')) and not ('Weights' in k)]
houses_encoded = houses_encoded.astype(
    {
        k:int for k in input_features_encoded
    }
)

#### show "houses_encoded"

In [ ]:
houses_encoded.info()

In [ ]:
houses_encoded.corr()['Price'].head(50)

In [ ]:
plt.figure(figsize = (25,25))
sns.heatmap(houses_encoded.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.show()

# Model Selection

## Data (for Modeling) & Data_Unseen (for test)

In [ ]:
data = stratified_sampling(
    df=houses_encoded,
    colname="PPDCategoryType",
    frac=0.95,
    random_state=1
)
data_unseen = houses_encoded.drop(data.index)
# drop index
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

## Select Data_train

In [ ]:
train_features = [k for k in data.columns if not ('Price' in k)] #or ('Month' in k))]
target = 'Price'
# inputs = [*train_features,target]
inputs = [
    'OldNew' ,  
    'Duration' , 
    'PPDCategoryType',
    # Month
    *[k for k in data.columns if ('Month' in k)],
    # Weights
    # *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    # # zscore
    *[k for k in data.columns if ('WeightsTransf' in k)],
    # PropertyType
    *[k for k in data.columns if ('PropertyType' in k)],
    # score
    *[i for i in data.columns if 'income' in i],
    # target
    target
]
data_train = data.loc[:,inputs]

In [ ]:
print(data_train.columns)
print(data_train.shape)

### Modeling

#### Option 1: Sklearn

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
inputs = [
    'OldNew' ,  
    'Duration' , 
    'PPDCategoryType',
    # Month
    *[k for k in data.columns if ('Month' in k)],
    # Weights
    *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    # # zscore
    # *[k for k in data.columns if ('WeightsTransf' in k)],
    # PropertyType
    *[k for k in data.columns if ('PropertyType' in k)],
    # score
    'income_area',
    # 'income_area_zscore',
    # *[i for i in data.columns if 'income' in i],
    # target
    target
]
data = houses_encoded.loc[:,inputs]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.loc[:,[i for i in data.columns if i!='Price']], 
    data.Price, 
    test_size=0.1, 
    random_state=13
)

##### HistGradientBoostingRegressor

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
model = HistGradientBoostingRegressor().fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score
y_predictions = model.predict(X_test)
r2_score(y_test, y_predictions)

##### xgboost

###### option 1:

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
model = XGBRegressor(
    n_estimators=1000, 
    max_depth=7, 
    eta=0.1, 
    subsample=0.7, 
    colsample_bytree=0.8,
    device="cuda"
)
cv = RepeatedKFold(
    n_splits=10, 
    n_repeats=3, 
    random_state=1
)
scores = cross_val_score(
    model, 
    X_train, 
    y_train, 
    scoring= 'r2',  #'neg_mean_squared_error', #'r2', 
    cv=cv, 
    n_jobs=-1
)
model.fit(X_train, y_train)
model

In [ ]:
model.score(X_train, y_train)

In [ ]:
y_predictions = model.predict(X_test)
r2_score(y_test, y_predictions)

In [ ]:
filename = model_dir_location + 'model.sav'

In [ ]:
# save the model to disk
pickle.dump(model, open(filename, 'wb'))

In [ ]:
order_features = [
    'OldNew', 
    'Duration', 
    'PPDCategoryType', 
    'TransferMonth_1', 
    'TransferMonth_2', 
    'TransferMonth_3', 
    'TransferMonth_4', 
    'TransferMonth_5', 
    'TransferMonth_6', 
    'TransferMonth_7', 
    'TransferMonth_8', 
    'TransferMonth_9', 
    'TransferMonth_10', 
    'TransferMonth_11', 
    'TransferMonth_12', 
    'CountyWeights', 
    'DistrictWeights', 
    'TownCityWeights', 
    'PostcodeAreaWeights', 
    'PropertyType_Detached', 
    'PropertyType_Flat', 
    'PropertyType_Other', 
    'PropertyType_SemiDetached', 
    'PropertyType_Terraced',
    'income_area'
] 

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test.loc[:,order_features], y_test)
print(result)

###### test

In [ ]:
X_test.iloc[0].tolist()

In [ ]:
input_tst= {
    'OldNew' : "Old", 
    'Duration': "Freehold", 
    'PPDCategoryType': "AdditionalPrice", 
    "Month" : 1,
    'CountyWeights': 2.61,
    'DistrictWeights':2.246, 
    'TownCityWeights':2.015, 
    'PostcodeAreaWeights':1.85,
    "PropertyType" : "Flat",
    "income_area": 44363.99
}
feature_oldnew =  OldNew_encoder.transform([input_tst['OldNew']])
feature_PPDCategoryType =  PPDCategoryType_encoder.transform([input_tst['PPDCategoryType']])
feature_Duration =  Duration_encoder.transform([input_tst['Duration']])
feature_PropertyType = PropertyType_ohe.transform([[input_tst['PropertyType']]])
feature_Month = Month_ohe.transform([[input_tst['Month']]])

In [ ]:
feature_Month

In [ ]:
features = [
    *feature_oldnew,
    *feature_Duration,
    *feature_PPDCategoryType,
    *feature_Month[0],
    *[input_tst['CountyWeights'],input_tst['DistrictWeights'],input_tst['TownCityWeights'],input_tst['PostcodeAreaWeights']],
    *feature_PropertyType[0],
    *[input_tst['income_area']]
]
features

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
y_predictions = loaded_model.predict([features])
y_predictions[0]

###### option 2:

In [ ]:
from sklearn.metrics import get_scorer_names
get_scorer_names()

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score, RandomizedSearchCV
model = XGBRegressor(
    n_estimators=1000, 
    max_depth=7, 
    eta=0.1, 
    subsample=0.7, 
    colsample_bytree=0.8,
    device="cuda"
)
log_params = {'penalty': ['l1', 'l2'], 'C': [1E-7, 1E-6, 1E-6, 1E-4, 1E-3]}
search = RandomizedSearchCV(
    model, 
    scoring='average_precision', 
    cv=10,
    n_iter=10, 
    param_distributions=log_params,
    refit=True, 
    n_jobs=-1
)
search.fit(X_train, y_train)
model = search.best_estimator_

##### linear regression:

In [ ]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

In [ ]:
lr_model.score(X_train, y_train)

In [ ]:
y_predictions = lr_model.predict(X_test)
r2_score(y_test, y_predictions)

##### SVM:

In [ ]:
from sklearn import svm
svm_model = svm.SVR()
svm_model.fit(X_train, y_train)

In [ ]:
svm_model.score(X_train, y_train)

In [ ]:
y_predictions = svm_model.predict(X_test)
r2_score(y_test, y_predictions)

#### Option 2: Pycaret

##### Training

In [ ]:
from pycaret.regression import *
predictor = setup(
    data = data_train, 
    target = 'Price', 
    normalize=True ,
    session_id=123,
    use_gpu=True
)

In [ ]:
models()

In [ ]:
best_model = compare_models()

In [ ]:
print(best_model)

##### plot

In [ ]:
plot_model(best_model, plot = 'residuals')

In [ ]:
plot_model(best_model, plot = 'error')

In [ ]:
plot_model(best_model, plot = 'feature')

##### Model tuning

In [ ]:
best_model_tuned = tune_model(best_model,optimize='R2')

##### predict tuned model

In [ ]:
best_model_tuned

In [ ]:
[x for x in data_unseen.columns]

In [ ]:
data_unseen_inputs = data_unseen.loc[:,[k for k in data_unseen.columns if k!='Price']]
data_unseen_output = data_unseen.loc[:,target]

In [ ]:
predictions = predict_model(best_model_tuned, data = data_unseen_inputs)

In [ ]:
predictions.head()

##### r2

In [ ]:
from sklearn.metrics import r2_score

r2_score(data_unseen_output, predictions['prediction_label'])

##### finalise model

In [ ]:
house_predictor_model = finalize_model(best_model_tuned)

In [ ]:
predictions_finalised = predict_model(house_predictor_model, data = data_unseen_inputs)

In [ ]:
from sklearn.metrics import r2_score

r2_score(data_unseen_output, predictions_finalised['prediction_label'])

##### save model

In [ ]:
model_version = 'v2' 

In [ ]:
name_model = model_dir_location + 'house_predictor_model_withIncomeArea' + '_' + model_version
save_model(house_predictor_model, name_model)

##### load model

In [ ]:
model_version = 'v2' 

In [ ]:
name_model = model_dir_location + 'house_predictor_model' + "_" + model_version
inputs = [
    'OldNew' ,  
    'Duration' , 
    'PPDCategoryType',
    # Month
    *[k for k in data.columns if ('Month' in k)],
    # Weights
    *[k for k in data.columns if ('Weights' in k) and (not ("Transf" in k))],
    # # zscore
    *[k for k in data.columns if ('WeightsTransf' in k)],
    # PropertyType
    *[k for k in data.columns if ('PropertyType' in k)],
    # target
    target
]
data_unseen_inputs = data_unseen.loc[:,[k for k in inputs if k!='Price']]

In [ ]:
house_predictor_model = load_model(name_model)

In [ ]:
predictions_mloaded = predict_model(house_predictor_model, data = data_unseen_inputs)

In [ ]:
from sklearn.metrics import r2_score

r2_score(data_unseen_output, predictions_mloaded['prediction_label'])

### Selecting Features
<a name='GDF'></a>
Above, we knew that an $x^2$ term was required. It may not always be obvious which features are required. One could add a variety of potential features to try and find the most useful. For example, what if we had instead tried : $y=w_0x_0 + w_1x_1^2 + w_2x_2^3+b$ ? 

Run the next cells. 